# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ushuaia,-54.8000,-68.3000,35.26,80,100,20.71,AR,1698614031
1,1,san antonio de pale,-1.4014,5.6325,79.83,82,5,9.19,GQ,1698614031
2,2,adamstown,-25.0660,-130.1015,72.21,84,98,14.67,PN,1698614031
3,3,rouissat,31.9243,5.3502,80.67,17,40,6.91,DZ,1698614031
4,4,utrik,11.2278,169.8474,82.90,81,35,18.75,MH,1698614031


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
cities_map = city_data_df.hvplot.points("Lng", 
                                      "Lat", 
                                      geo = True,
                                      color = "City",
                                      alpha = 0.6,
                                      size = "Humidity",
                                      tiles = "OSM"
                                     )

# Display the map
cities_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values


weather_df = city_data_df.loc[(city_data_df["Max Temp"] < 85)
                                       & (city_data_df["Max Temp"] > 50) 
                                       & (city_data_df["Cloudiness"] < 2)
                                       & (city_data_df["Wind Speed"] < 3)
                                      ]

# Drop any rows with null values
weather_df.dropna()

# Display sample data
weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
139,139,prineville,44.2999,-120.8345,53.49,39,0,1.01,US,1698614058
179,179,fortuna,40.5982,-124.1573,72.93,46,0,1.01,US,1698614067
199,199,touba,8.2833,-7.6833,74.84,68,0,2.04,CI,1698614071
353,353,farah,32.5000,62.5000,67.26,34,0,2.48,AF,1698614108
468,468,bageshwar,29.8500,79.7667,55.49,72,0,2.68,IN,1698614140
564,564,bulancak,40.9372,38.2291,60.62,88,0,0.00,TR,1698614159


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
139,prineville,US,44.2999,-120.8345,39,
179,fortuna,US,40.5982,-124.1573,46,
199,touba,CI,8.2833,-7.6833,68,
353,farah,AF,32.5000,62.5000,34,
468,bageshwar,IN,29.8500,79.7667,72,
564,bulancak,TR,40.9372,38.2291,88,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
params = {'categories': 'accommodation.hotel',
          'apiKey':geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
prineville - nearest hotel: Best Western
fortuna - nearest hotel: Comfort Inn & Suites Redwood Country
touba - nearest hotel: No hotel found
farah - nearest hotel: No hotel found
bageshwar - nearest hotel: vivek hotel
bulancak - nearest hotel: elit otel pansiyon


,City,Country,Lat,Lng,Humidity,Hotel Name
139,prineville,US,44.2999,-120.8345,39,Best Western
179,fortuna,US,40.5982,-124.1573,46,Comfort Inn & Suites Redwood Country
199,touba,CI,8.2833,-7.6833,68,No hotel found
353,farah,AF,32.5000,62.5000,34,No hotel found
468,bageshwar,IN,29.8500,79.7667,72,vivek hotel
564,bulancak,TR,40.9372,38.2291,88,elit otel pansiyon


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 0.8,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"]
                                  )

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)